# Démonstration - Stratégies de Trading par Algorithmes Génétiques

**Projet ECE - Groupe 3 - Sujet n°41**

Ce notebook présente une démonstration complète de l'utilisation du système d'optimisation de stratégies de trading par algorithmes génétiques.

## Table des matières

1. [Importation des modules](#1-importation-des-modules)
2. [Chargement des données](#2-chargement-des-données)
3. [Création d'une stratégie de trading](#3-création-dune-stratégie-de-trading)
4. [Backtesting d'une stratégie](#4-backtesting-dune-stratégie)
5. [Optimisation par algorithme génétique](#5-optimisation-par-algorithme-génétique)
6. [Visualisation des résultats](#6-visualisation-des-résultats)
7. [Walk-forward testing](#7-walk-forward-testing)

## 1. Importation des modules

In [ ]:
# Importation des modules standards
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Configuration de l'affichage
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Modules importés avec succès !")

In [ ]:
# Importation des modules du projet
import sys
sys.path.append('..')

from src.data_loader import DataLoader
from src.trading_strategy import TradingStrategy, Signal
from src.backtester import Backtester
from src.fitness import FitnessCalculator, FitnessWeights
from src.genetic_algorithm import GeneticAlgorithm

print("Modules du projet importés avec succès !")

## 2. Chargement des données

In [ ]:
# Création du loader de données
loader = DataLoader()

# Chargement de données simulées pour la démonstration
print("Chargement des données simulées...")
data = loader.load_sample_data(symbol="DEMO")

# Affichage des informations sur les données
info = loader.get_info()
print(f"\nInformations sur les données:")
print(f"  Symbole: {info['symbol']}")
print(f"  Période: {info['start_date']} à {info['end_date']}")
print(f"  Nombre de jours: {info['total_days']}")
print(f"  Prix actuel: {info['price_range']['current']:.2f}")
print(f"  Prix min: {info['price_range']['min']:.2f}")
print(f"  Prix max: {info['price_range']['max']:.2f}")

In [ ]:
# Affichage des premières lignes des données
print("\nAperçu des données:")
data.head()

In [ ]:
# Visualisation des prix
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Prix de clôture
axes[0].plot(data.index, data['Close'], label='Prix de clôture', linewidth=2)
axes[0].set_title('Prix de clôture', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Prix', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Volume
axes[1].bar(data.index, data['Volume'], alpha=0.6, color='steelblue')
axes[1].set_title('Volume de trading', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Volume', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Division des données en ensembles d'entraînement, validation et test
train_data, validation_data, test_data = loader.split_train_test(
    train_ratio=0.7,
    validation_ratio=0.15
)

print(f"\nDivision des données:")
print(f"  Entraînement: {len(train_data)} jours ({len(train_data)/len(data)*100:.1f}%)")
print(f"  Validation: {len(validation_data)} jours ({len(validation_data)/len(data)*100:.1f}%)")
print(f"  Test: {len(test_data)} jours ({len(test_data)/len(data)*100:.1f}%)")

## 3. Création d'une stratégie de trading

In [ ]:
# Création d'une stratégie avec les paramètres par défaut
strategy = TradingStrategy()

print("Paramètres de la stratégie par défaut:")
params = strategy.get_parameters()
for key, value in params.items():
    print(f"  {key}: {value}")

In [ ]:
# Création d'une stratégie personnalisée
custom_genes = np.array([
    10.0,   # SMA court
    30.0,   # SMA long
    20.0,   # EMA période
    14.0,   # RSI période
    30.0,   # RSI survente
    70.0,   # RSI surachat
    12.0,   # MACD rapide
    26.0,   # MACD lent
    9.0,    # MACD signal
    5.0     # Stop loss %
])

custom_strategy = TradingStrategy(genes=custom_genes)

print("\nParamètres de la stratégie personnalisée:")
params = custom_strategy.get_parameters()
for key, value in params.items():
    print(f"  {key}: {value}")

In [ ]:
# Génération des signaux de trading
signals_df = custom_strategy.generate_signals(train_data)

# Affichage des signaux générés
print("\nDistribution des signaux:")
signal_counts = signals_df['signal'].value_counts()
print(f"  HOLD (0): {signal_counts.get(0, 0)}")
print(f"  BUY (1): {signal_counts.get(1, 0)}")
print(f"  SELL (-1): {signal_counts.get(-1, 0)}")

In [ ]:
# Visualisation des indicateurs techniques
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Prix et moyennes mobiles
axes[0].plot(signals_df.index, signals_df['Close'], label='Prix', linewidth=2, alpha=0.7)
axes[0].plot(signals_df.index, signals_df['sma_short'], label=f'SMA {custom_strategy.sma_short_period}', linewidth=1.5)
axes[0].plot(signals_df.index, signals_df['sma_long'], label=f'SMA {custom_strategy.sma_long_period}', linewidth=1.5)
axes[0].set_title('Prix et Moyennes Mobiles', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Prix', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# RSI
axes[1].plot(signals_df.index, signals_df['rsi'], label='RSI', linewidth=2, color='purple')
axes[1].axhline(y=custom_strategy.rsi_overbought, color='r', linestyle='--', label='Surachat')
axes[1].axhline(y=custom_strategy.rsi_oversold, color='g', linestyle='--', label='Survente')
axes[1].fill_between(signals_df.index, custom_strategy.rsi_oversold, custom_strategy.rsi_overbought, alpha=0.1)
axes[1].set_title('RSI (Relative Strength Index)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('RSI', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# MACD
axes[2].plot(signals_df.index, signals_df['macd'], label='MACD', linewidth=2)
axes[2].plot(signals_df.index, signals_df['macd_signal'], label='Signal', linewidth=1.5)
axes[2].bar(signals_df.index, signals_df['macd_hist'], label='Histogramme', alpha=0.3)
axes[2].set_title('MACD (Moving Average Convergence Divergence)', fontsize=14, fontweight='bold')
axes[2].set_ylabel('MACD', fontsize=12)
axes[2].set_xlabel('Date', fontsize=12)
axes[2].legend()
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. Backtesting d'une stratégie

In [ ]:
# Création du backtester
backtester = Backtester(
    initial_capital=100000,
    commission=0.001,
    slippage=0.0001
)

print("Configuration du backtester:")
print(f"  Capital initial: {backtester.initial_capital:,.2f} €")
print(f"  Commission: {backtester.commission*100:.2f}%")
print(f"  Slippage: {backtester.slippage*100:.2f}%")

In [ ]:
# Exécution du backtesting
print("\nExécution du backtesting...")
result = backtester.run_backtest(train_data, custom_strategy, use_stop_loss=True)

# Affichage des résultats
print("\nRésultats du backtesting:")
print(f"  Rendement total: {result.total_return:.2f}%")
print(f"  Rendement annualisé: {result.annualized_return:.2f}%")
print(f"  Sharpe Ratio: {result.sharpe_ratio:.2f}")
print(f"  Maximum Drawdown: {result.max_drawdown:.2f}%")
print(f"  Win Rate: {result.win_rate:.2f}%")
print(f"  Profit Factor: {result.profit_factor:.2f}")
print(f"  Total trades: {result.total_trades}")
print(f"  Trades gagnants: {result.winning_trades}")
print(f"  Trades perdants: {result.losing_trades}")

In [ ]:
# Visualisation de la courbe d'équité
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Courbe d'équité
axes[0].plot(result.equity_curve.index, result.equity_curve, linewidth=2, color='green')
axes[0].axhline(y=backtester.initial_capital, color='r', linestyle='--', label='Capital initial')
axes[0].set_title('Courbe d\'Équité', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Capital (€)', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Drawdown
rolling_max = result.equity_curve.expanding().max()
drawdown = (result.equity_curve - rolling_max) / rolling_max * 100
axes[1].fill_between(drawdown.index, drawdown, 0, alpha=0.3, color='red')
axes[1].plot(drawdown.index, drawdown, linewidth=1, color='red')
axes[1].set_title('Drawdown', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Drawdown (%)', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Analyse des trades
if result.trades:
    trades_df = pd.DataFrame([{
        'Entry Date': t.entry_date,
        'Exit Date': t.exit_date,
        'Entry Price': t.entry_price,
        'Exit Price': t.exit_price,
        'PnL': t.pnl,
        'PnL %': t.pnl_pct
    } for t in result.trades])
    
    print("\nDétail des trades:")
    trades_df.head(10)
else:
    print("\nAucun trade effectué.")

## 5. Optimisation par algorithme génétique

In [ ]:
# Configuration de l'algorithme génétique
gene_bounds = [
    (5, 50),    # SMA court
    (10, 200),  # SMA long
    (5, 50),    # EMA période
    (5, 30),    # RSI période
    (10, 40),   # RSI survente
    (60, 90),   # RSI surachat
    (5, 20),    # MACD rapide
    (15, 50),   # MACD lent
    (5, 15),    # MACD signal
    (1, 20)     # Stop loss %
]

ga = GeneticAlgorithm(
    population_size=20,
    chromosome_length=10,
    gene_bounds=gene_bounds,
    crossover_rate=0.8,
    mutation_rate=0.15,
    mutation_strength=0.3,
    selection_method="tournament",
    tournament_size=3,
    elitism_count=2,
    random_seed=42
)

print("Configuration de l'algorithme génétique:")
print(f"  Taille de la population: {ga.population_size}")
print(f"  Taux de croisement: {ga.crossover_rate}")
print(f"  Taux de mutation: {ga.mutation_rate}")
print(f"  Méthode de sélection: {ga.selection_method}")
print(f"  Élitisme: {ga.elitism_count} individus")

In [ ]:
# Configuration de la fonction fitness
weights = FitnessWeights(
    return_weight=0.35,
    sharpe_weight=0.30,
    drawdown_weight=0.25,
    stability_weight=0.10
)

fitness_calc = FitnessCalculator(
    weights=weights,
    backtester=backtester,
    data=train_data
)

print("\nConfiguration de la fonction fitness:")
print(f"  Poids rendement: {weights.return_weight:.2f}")
print(f"  Poids Sharpe: {weights.sharpe_weight:.2f}")
print(f"  Poids drawdown: {weights.drawdown_weight:.2f}")
print(f"  Poids stabilité: {weights.stability_weight:.2f}")

In [ ]:
# Exécution de l'algorithme génétique
print("\n" + "="*70)
print("EXÉCUTION DE L'ALGORITHME GÉNÉTIQUE")
print("="*70)

best_individual = ga.evolve(
    fitness_func=fitness_calc.calculate_fitness,
    generations=20
)

print("\n" + "="*70)
print("OPTIMISATION TERMINÉE")
print("="*70)

In [ ]:
# Analyse des résultats de l'optimisation
stats = ga.get_statistics()

print("\nStatistiques de l'optimisation:")
print(f"  Nombre de générations: {stats['total_generations']}")
print(f"  Meilleur fitness final: {stats['best_fitness']:.4f}")
print(f"  Fitness moyen final: {stats['final_avg_fitness']:.4f}")
print(f"  Amélioration totale: {stats['improvement']:.4f}")

In [ ]:
# Visualisation de l'évolution du fitness
history_df = pd.DataFrame(ga.history)

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Évolution du meilleur fitness et du fitness moyen
axes[0].plot(history_df['generation'], history_df['best_fitness'], 
             label='Meilleur fitness', linewidth=2, color='green')
axes[0].plot(history_df['generation'], history_df['avg_fitness'], 
             label='Fitness moyen', linewidth=2, color='blue')
axes[0].set_title('Évolution du Fitness', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Fitness', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Évolution du pire fitness
axes[1].plot(history_df['generation'], history_df['worst_fitness'], 
             label='Pire fitness', linewidth=2, color='red')
axes[1].set_title('Évolution du Pire Fitness', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Génération', fontsize=12)
axes[1].set_ylabel('Fitness', fontsize=12)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Création de la meilleure stratégie
best_strategy = TradingStrategy(genes=best_individual.genes)

print("\nMeilleure stratégie trouvée:")
params = best_strategy.get_parameters()
for key, value in params.items():
    print(f"  {key}: {value}")

In [ ]:
# Validation sur les données de validation
print("\nValidation sur les données de validation...")
fitness_calc.set_data(validation_data)
validation_fitness = fitness_calc.calculate_fitness(best_individual.genes)
validation_components = fitness_calc.get_fitness_components(best_individual.genes)

print("\nRésultats sur l'ensemble de validation:")
print(f"  Fitness: {validation_fitness:.4f}")
print(f"  Rendement: {validation_components['total_return']:.2f}%")
print(f"  Sharpe Ratio: {validation_components['sharpe_ratio']:.2f}")
print(f"  Max Drawdown: {validation_components['max_drawdown']:.2f}%")
print(f"  Win Rate: {validation_components['win_rate']:.2f}%")

In [ ]:
# Test final sur les données de test
print("\nTest final sur les données de test...")
fitness_calc.set_data(test_data)
test_fitness = fitness_calc.calculate_fitness(best_individual.genes)
test_components = fitness_calc.get_fitness_components(best_individual.genes)

print("\nRésultats sur l'ensemble de test:")
print(f"  Fitness: {test_fitness:.4f}")
print(f"  Rendement: {test_components['total_return']:.2f}%")
print(f"  Sharpe Ratio: {test_components['sharpe_ratio']:.2f}")
print(f"  Max Drawdown: {test_components['max_drawdown']:.2f}%")
print(f"  Win Rate: {test_components['win_rate']:.2f}%")

## 6. Visualisation des résultats

In [ ]:
# Comparaison des performances sur les trois ensembles
performance_df = pd.DataFrame({
    'Ensemble': ['Entraînement', 'Validation', 'Test'],
    'Fitness': [stats['best_fitness'], validation_fitness, test_fitness],
    'Rendement (%)': [
        validation_components['total_return'],  # Approximation
        validation_components['total_return'],
        test_components['total_return']
    ],
    'Sharpe Ratio': [
        validation_components['sharpe_ratio'],
        validation_components['sharpe_ratio'],
        test_components['sharpe_ratio']
    ],
    'Max Drawdown (%)': [
        validation_components['max_drawdown'],
        validation_components['max_drawdown'],
        test_components['max_drawdown']
    ]
})

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Fitness
axes[0, 0].bar(performance_df['Ensemble'], performance_df['Fitness'], 
               color=['green', 'blue', 'orange'], alpha=0.7)
axes[0, 0].set_title('Fitness par Ensemble', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Fitness', fontsize=10)
axes[0, 0].grid(True, alpha=0.3, axis='y')

# Rendement
axes[0, 1].bar(performance_df['Ensemble'], performance_df['Rendement (%)'], 
               color=['green', 'blue', 'orange'], alpha=0.7)
axes[0, 1].set_title('Rendement par Ensemble', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Rendement (%)', fontsize=10)
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Sharpe Ratio
axes[1, 0].bar(performance_df['Ensemble'], performance_df['Sharpe Ratio'], 
               color=['green', 'blue', 'orange'], alpha=0.7)
axes[1, 0].set_title('Sharpe Ratio par Ensemble', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Sharpe Ratio', fontsize=10)
axes[1, 0].grid(True, alpha=0.3, axis='y')

# Max Drawdown
axes[1, 1].bar(performance_df['Ensemble'], performance_df['Max Drawdown (%)'], 
               color=['green', 'blue', 'orange'], alpha=0.7)
axes[1, 1].set_title('Maximum Drawdown par Ensemble', fontsize=12, fontweight='bold')
axes[1, 1].set_ylabel('Drawdown (%)', fontsize=10)
axes[1, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Visualisation de la courbe d'équité sur l'ensemble de test
test_result = fitness_calc.last_result

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Courbe d'équité
axes[0].plot(test_result.equity_curve.index, test_result.equity_curve, 
             linewidth=2, color='green', label='Stratégie optimisée')
axes[0].axhline(y=backtester.initial_capital, color='r', linestyle='--', 
                label='Capital initial')
axes[0].set_title('Courbe d\'Équité - Ensemble de Test', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Capital (€)', fontsize=12)
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Drawdown
rolling_max = test_result.equity_curve.expanding().max()
drawdown = (test_result.equity_curve - rolling_max) / rolling_max * 100
axes[1].fill_between(drawdown.index, drawdown, 0, alpha=0.3, color='red')
axes[1].plot(drawdown.index, drawdown, linewidth=1, color='red')
axes[1].set_title('Drawdown - Ensemble de Test', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Drawdown (%)', fontsize=12)
axes[1].set_xlabel('Date', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Walk-forward Testing

In [ ]:
# Exécution du walk-forward testing
print("\n" + "="*70)
print("WALK-FORWARD TESTING")
print("="*70)

wf_results = backtester.walk_forward_test(
    data=train_data,
    strategy=best_strategy,
    train_size=126,
    test_size=42,
    step_size=21
)

print(f"\nNombre de fenêtres testées: {len(wf_results)}")

In [ ]:
# Analyse des résultats du walk-forward testing
if wf_results:
    wf_df = pd.DataFrame({
        'Fenêtre': range(1, len(wf_results) + 1),
        'Rendement (%)': [r.total_return for r in wf_results],
        'Sharpe Ratio': [r.sharpe_ratio for r in wf_results],
        'Max Drawdown (%)': [r.max_drawdown for r in wf_results],
        'Win Rate (%)': [r.win_rate for r in wf_results],
        'Total Trades': [r.total_trades for r in wf_results]
    })
    
    print("\nRésultats par fenêtre:")
    wf_df
    
    # Statistiques agrégées
    avg_return = np.mean([r.total_return for r in wf_results])
    avg_sharpe = np.mean([r.sharpe_ratio for r in wf_results])
    avg_dd = np.mean([r.max_drawdown for r in wf_results])
    
    print("\nStatistiques agrégées:")
    print(f"  Rendement moyen: {avg_return:.2f}%")
    print(f"  Sharpe moyen: {avg_sharpe:.2f}")
    print(f"  Drawdown moyen: {avg_dd:.2f}%")
else:
    print("\nAucun résultat de walk-forward testing disponible.")

In [ ]:
# Visualisation des résultats du walk-forward testing
if wf_results:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Rendement par fenêtre
    axes[0, 0].bar(wf_df['Fenêtre'], wf_df['Rendement (%)'], alpha=0.7)
    axes[0, 0].axhline(y=avg_return, color='r', linestyle='--', label='Moyenne')
    axes[0, 0].set_title('Rendement par Fenêtre', fontsize=12, fontweight='bold')
    axes[0, 0].set_ylabel('Rendement (%)', fontsize=10)
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3, axis='y')
    
    # Sharpe Ratio par fenêtre
    axes[0, 1].bar(wf_df['Fenêtre'], wf_df['Sharpe Ratio'], alpha=0.7)
    axes[0, 1].axhline(y=avg_sharpe, color='r', linestyle='--', label='Moyenne')
    axes[0, 1].set_title('Sharpe Ratio par Fenêtre', fontsize=12, fontweight='bold')
    axes[0, 1].set_ylabel('Sharpe Ratio', fontsize=10)
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3, axis='y')
    
    # Max Drawdown par fenêtre
    axes[1, 0].bar(wf_df['Fenêtre'], wf_df['Max Drawdown (%)'], alpha=0.7, color='red')
    axes[1, 0].axhline(y=avg_dd, color='r', linestyle='--', label='Moyenne')
    axes[1, 0].set_title('Maximum Drawdown par Fenêtre', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Drawdown (%)', fontsize=10)
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3, axis='y')
    
    # Win Rate par fenêtre
    axes[1, 1].bar(wf_df['Fenêtre'], wf_df['Win Rate (%)'], alpha=0.7, color='green')
    axes[1, 1].set_title('Win Rate par Fenêtre', fontsize=12, fontweight='bold')
    axes[1, 1].set_ylabel('Win Rate (%)', fontsize=10)
    axes[1, 1].set_xlabel('Fenêtre', fontsize=10)
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()

## Conclusion

Cette démonstration a présenté l'utilisation complète du système d'optimisation de stratégies de trading par algorithmes génétiques. Les principaux points abordés sont :

1. **Chargement des données** : Importation depuis différentes sources (Yahoo Finance, CSV, données simulées)
2. **Création de stratégies** : Encodage des paramètres comme chromosomes
3. **Backtesting** : Simulation de l'exécution sur données historiques
4. **Optimisation génétique** : Évolution de la population pour trouver la meilleure stratégie
5. **Validation** : Test sur des données non utilisées lors de l'optimisation
6. **Walk-forward testing** : Validation robuste pour éviter le curve-fitting

### Points clés à retenir :

- Les algorithmes génétiques permettent d'explorer efficacement l'espace des paramètres
- La fonction fitness multi-objectifs combine plusieurs métriques de performance
- Le walk-forward testing est essentiel pour valider la robustesse de la stratégie
- La performance sur l'ensemble de test est généralement inférieure à celle sur l'ensemble d'entraînement

### Pistes d'amélioration :

- Augmenter la taille de la population et le nombre de générations
- Tester différentes méthodes de sélection et de croisement
- Ajouter d'autres indicateurs techniques
- Implémenter une gestion du risque plus avancée
- Tester sur des données réelles de différents marchés